In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
#Libraries
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from google.colab import files
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
movie_data = pd.read_csv('/content/IMDb_movies.csv', header=0)
# Only English movies 
movie_data = movie_data.drop(movie_data.index[movie_data['language'] !='English'])
# use "title", "genre", "descrition" only
movie_data = movie_data.drop(["language","imdb_title_id", "year","original_title", "date_published","duration", "country", "writer", "production_company", "actors", "avg_vote", "director", "votes", "budget", "usa_gross_income", "worlwide_gross_income", "metascore","reviews_from_users", "reviews_from_critics"], axis=1)
print(movie_data.shape)
movie_data.head(20)

In [ ]:
# use only one genre for classification
genres = []
index = 0
for movie in movie_data['genre']:
  if re.match(".*,.*", movie):
    genres = movie.split(", ")
    for i in range(len(genres)):
      # See if sci-fi or fantasy is included in the genre list
      if genres[i] == 'Fantasy' or genres[i] == 'Sci-Fi':
        movie_data.iloc[index]['genre'] = genres[i]
      elif i == len(genres)-1:
        movie_data.iloc[index]['genre'] = genres[i]
      else:
        continue
          

  index += 1

movie_data = movie_data[(movie_data.genre == 'Fantasy') | (movie_data[:17000].genre == 'Sci-Fi')]

In [ ]:
movie_data.genre.unique()

In [ ]:
# code from https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lem = WordNetLemmatizer()
def lemmaSentence(sentence):
    token_words=word_tokenize(sentence)
    lemma_sentence = [lem.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
    return " ".join(lemma_sentence)

index = 0
for movie in movie_data['description']:
  movie_data.iloc[index]['description'] = lemmaSentence(str(movie_data.iloc[index]['description']))
  index += 1

In [ ]:
print(movie_data['genre'].shape)
movie_data['genre'].value_counts()

In [ ]:
# Change data into a Document-Term Matrix(DTM)
data = movie_data['description'].values.astype('U')
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
vectorizer = CountVectorizer(stop_words = 'english', strip_accents='unicode', tokenizer = token.tokenize)
X = vectorizer.fit_transform(data)
#print(vectorizer.get_feature_names())
# save as csv for student use
pd.DataFrame(X.toarray()).to_csv("movie_descriptions.csv", header=vectorizer.get_feature_names())
files.download("movie_descriptions.csv")

#print(movie_data['genre'])
movie_data['genre'].to_csv('movie_genres.csv', index=False)
files.download("movie_genres.csv")